In [ ]:
#Mineria de datos
#Laboratorio 2
#Anggelie Velasquez 22
#Anthony Lou 23410
#Isabella Obando 23074
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
from scipy.spatial.distance import pdist
from sklearn.neighbors import NearestNeighbors

import warnings
warnings.filterwarnings("ignore")

In [4]:
df = pd.read_csv("movies_2026.csv", encoding="latin1")
df.head()

,id,budget,genres,homePage,productionCompany,productionCompanyCountry,productionCountry,revenue,runtime,video,...,releaseDate,voteAvg,voteCount,genresAmount,productionCoAmount,productionCountriesAmount,actorsAmount,castWomenAmount,castMenAmount,releaseYear
0,1627085,0.0,Drama|Crime,NaN,NaN,NaN,NaN,0.0,95,False,...,2026-02-01,0.0,0,2,0,0,8,2.0,5.0,2026.0
1,1626914,0.0,Animation,NaN,NaN,NaN,NaN,0.0,3,False,...,2026-02-01,0.0,0,1,0,0,4,0.0,0.0,2026.0
2,1626898,0.0,Animation,NaN,NaN,NaN,NaN,0.0,2,False,...,2026-02-01,0.0,0,1,0,0,3,0.0,0.0,2026.0
3,1626808,0.0,Thriller|Mystery|Documentary,NaN,NaN,NaN,NaN,0.0,5,False,...,2026-02-01,0.0,0,3,0,0,7,0.0,0.0,2026.0
4,1626678,0.0,Animation,NaN,NaN,NaN,NaN,0.0,12,False,...,2026-02-01,0.0,0,1,0,0,3,0.0,0.0,2026.0


In [9]:
df.columns


Index(['id', 'budget', 'genres', 'homePage', 'productionCompany',
       'productionCompanyCountry', 'productionCountry', 'revenue', 'runtime',
       'video', 'director', 'actors', 'actorsPopularity', 'actorsCharacter',
       'originalTitle', 'title', 'originalLanguage', 'popularity',
       'releaseDate', 'voteAvg', 'voteCount', 'genresAmount',
       'productionCoAmount', 'productionCountriesAmount', 'actorsAmount',
       'castWomenAmount', 'castMenAmount', 'releaseYear'],
      dtype='object')

In [11]:
df_cluster = df.drop(columns=[
    'id', 'genres', 'homePage', 'productionCompany',
    'productionCompanyCountry', 'productionCountry',
    'director', 'actors', 'actorsCharacter',
    'originalTitle', 'title', 'originalLanguage',
    'releaseDate', 'video'
])

In [12]:
df_cluster.isnull().sum()

budget                          0
revenue                         0
runtime                         0
actorsPopularity             2608
popularity                      0
voteAvg                         0
voteCount                       0
genresAmount                    0
productionCoAmount              0
productionCountriesAmount       0
actorsAmount                    0
castWomenAmount                37
castMenAmount                 162
releaseYear                     2
dtype: int64

In [16]:
df_cluster = df_cluster.select_dtypes(include=['int64','float64'])

In [18]:
df_cluster = df_cluster.fillna(df_cluster.median())

In [19]:
df_cluster.isnull().sum()

budget                       0
revenue                      0
runtime                      0
popularity                   0
voteAvg                      0
voteCount                    0
genresAmount                 0
productionCoAmount           0
productionCountriesAmount    0
actorsAmount                 0
castWomenAmount              0
castMenAmount                0
releaseYear                  0
dtype: int64

In [21]:
(df_cluster['revenue'] == 0).sum()

14485

In [20]:
(df_cluster['budget'] == 0).sum()

13281

In [22]:
df_cluster = df_cluster.drop(columns=['budget','revenue'])
df_cluster.head()

,runtime,popularity,voteAvg,voteCount,genresAmount,productionCoAmount,productionCountriesAmount,actorsAmount,castWomenAmount,castMenAmount,releaseYear
0,95,0.0357,0.0,0,2,0,0,8,2.0,5.0,2026.0
1,3,0.0357,0.0,0,1,0,0,4,0.0,0.0,2026.0
2,2,0.0214,0.0,0,1,0,0,3,0.0,0.0,2026.0
3,5,0.0429,0.0,0,3,0,0,7,0.0,0.0,2026.0
4,12,0.0379,0.0,0,1,0,0,3,0.0,0.0,2026.0


In [23]:
df_cluster.dtypes

runtime                        int64
popularity                   float64
voteAvg                      float64
voteCount                      int64
genresAmount                   int64
productionCoAmount             int64
productionCountriesAmount      int64
actorsAmount                   int64
castWomenAmount              float64
castMenAmount                float64
releaseYear                  float64
dtype: object

In [26]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_cluster)

In [27]:
df_cluster.describe()

,runtime,popularity,voteAvg,voteCount,genresAmount,productionCoAmount,productionCountriesAmount,actorsAmount,castWomenAmount,castMenAmount,releaseYear
count,19883.000000,19883.000000,19883.000000,19883.000000,19883.000000,19883.000000,19883.000000,19883.000000,19883.000000,19883.000000,19883.000000
mean,66.093799,26.248842,3.837069,675.927325,1.949253,1.972539,1.230247,1082.049640,3509.965901,8157.367248,2016.827642
std,50.089055,156.067177,3.397705,1938.194656,1.256225,2.312075,2.244688,26402.818732,47724.433405,70970.890375,12.837427
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1902.000000
25%,10.000000,0.054600,0.000000,0.000000,1.000000,0.000000,1.000000,3.000000,0.000000,0.000000,2013.000000
50%,86.000000,8.502000,5.400000,6.000000,2.000000,1.000000,1.000000,9.000000,2.000000,3.000000,2021.000000
75%,103.000000,22.239000,6.800000,423.000000,3.000000,3.000000,1.000000,21.000000,6.000000,11.000000,2025.000000
max,750.000000,11474.647000,10.000000,30788.000000,16.000000,89.000000,155.000000,919590.000000,922162.000000,922017.000000,2026.000000


In [28]:
df_cluster = df_cluster.drop(columns=[
    'actorsAmount',
    'castWomenAmount',
    'castMenAmount'
])

Se eliminaron las columnas de budget y revenue ya que tenian muchos datos en 0, no se van a tomar en cuenta en el clustering 

In [29]:
df_cluster.describe()

,runtime,popularity,voteAvg,voteCount,genresAmount,productionCoAmount,productionCountriesAmount,releaseYear
count,19883.000000,19883.000000,19883.000000,19883.000000,19883.000000,19883.000000,19883.000000,19883.000000
mean,66.093799,26.248842,3.837069,675.927325,1.949253,1.972539,1.230247,2016.827642
std,50.089055,156.067177,3.397705,1938.194656,1.256225,2.312075,2.244688,12.837427
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1902.000000
25%,10.000000,0.054600,0.000000,0.000000,1.000000,0.000000,1.000000,2013.000000
50%,86.000000,8.502000,5.400000,6.000000,2.000000,1.000000,1.000000,2021.000000
75%,103.000000,22.239000,6.800000,423.000000,3.000000,3.000000,1.000000,2025.000000
max,750.000000,11474.647000,10.000000,30788.000000,16.000000,89.000000,155.000000,2026.000000


In [30]:
import numpy as np

df_cluster['popularity'] = np.log1p(df_cluster['popularity'])
df_cluster['voteCount'] = np.log1p(df_cluster['voteCount'])
df_cluster['productionCoAmount'] = np.log1p(df_cluster['productionCoAmount'])
df_cluster['productionCountriesAmount'] = np.log1p(df_cluster['productionCountriesAmount'])

In [31]:

(df_cluster['runtime'] == 0).sum()

2951

In [32]:
median_runtime = df_cluster.loc[df_cluster['runtime'] > 0, 'runtime'].median()
median_runtime

90.0

In [33]:
df_cluster.loc[df_cluster['runtime'] == 0, 'runtime'] = median_runtime

In [34]:
(df_cluster['runtime'] == 0).sum()


0

Se agregaron valores con la media a valores en 0 de runtime

In [35]:
zeros_count = (df_cluster == 0).sum()
zeros_count

runtime                         0
popularity                    707
voteAvg                      8326
voteCount                    8302
genresAmount                 1930
productionCoAmount           5241
productionCountriesAmount    3729
releaseYear                     0
dtype: int64

KeyError: 'voteAvg '